# Hand Position

In [1]:
import numpy as np
import pandas as pd


### Loading the data

In [2]:
spontan1 = pd.read_csv('Training/spontan/Evyatar636771052727603804Spontan.csv')
spontan2 = pd.read_csv('Training/spontan/Nofar636759795182793299Spontan.csv')
spontan3 = pd.read_csv('Training/spontan/Oriya636759804404113837Spontan.csv')
spontan4 = pd.read_csv('Training/spontan/OryaB636771083605535985Spontan.csv')
spontan5 = pd.read_csv('Training/spontan/Revital636770957038873193Spontan.csv')
spontan6 = pd.read_csv('Training/spontan/Sachar636771045591258071Spontan.csv')
spontan7 = pd.read_csv('Training/spontan/Shelly636759787508979097Spontan.csv')
spontan8 = pd.read_csv('Training/spontan/Yael636771059544754823Spontan.csv')
spontan9 = pd.read_csv('Training/spontan/Yoel636760921663075813Spontan.csv')

In [3]:
sync1 = pd.read_csv('Training/synchron/Evyatar636771053639929594Sync.csv')
sync2 = pd.read_csv('Training/synchron/Nofar636759797397919664Sync.csv')
sync3 = pd.read_csv('Training/synchron/Oriya636759806131350399Sync.csv')
sync4 = pd.read_csv('Training/synchron/OryaB636771084494502229Sync.csv')
sync5 = pd.read_csv('Training/synchron/Revital636770957894370062Sync.csv')
sync6 = pd.read_csv('Training/synchron/Sachar636771046527112972Sync.csv')
sync7 = pd.read_csv('Training/synchron/Shelly636759789526313406Sync.csv')
sync8 = pd.read_csv('Training/synchron/Yael636771060699529085Sync.csv')
sync9 = pd.read_csv('Training/synchron/Yoel636760923490742912Sync.csv')

In [4]:
alone1 = pd.read_csv('Training/alone/Evyatar636771054555711409Alone.csv')
alone2 = pd.read_csv('Training/alone/Nofar636759796290435160Alone.csv')
alone3 = pd.read_csv('Training/alone/Oriya636759805268396661Alone.csv')
alone4 = pd.read_csv('Training/alone/OryaB636771082736601528Alone.csv')
alone5 = pd.read_csv('Training/alone/Revital636770955999435879Alone.csv')
alone6 = pd.read_csv('Training/alone/Sachar636771047408504303Alone.csv')
alone7 = pd.read_csv('Training/alone/Shelly636759788383205274Alone.csv')
alone8 = pd.read_csv('Training/alone/Yael636771061659067649Alone.csv')
alone9 = pd.read_csv('Training/alone/Yoel636760922573364655Alone.csv')

In [5]:

spontan1[::10].head(10)

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,335.9984,128556,right,2,128.6357,224.0294,-12.29588,-18.262430,-140.54450,-117.57790,...,-1.588251,177.5125,173.5658,-14.569630,276.8753,-33.68264,91.61740,0.000000,1.250576,0.0
10,336.0816,128566,right,2,134.0850,214.4446,-28.93817,138.872500,-93.84897,-209.65120,...,-1.200900,181.0919,163.4637,-15.320050,268.2274,-43.06868,102.24200,0.000000,0.802890,0.0
20,336.1640,128576,right,2,144.6519,207.9939,-45.14541,98.476040,-157.00970,-164.59980,...,-0.702534,183.0549,159.0832,-10.880010,247.9993,-59.33919,99.31474,0.000000,0.727907,0.0
30,336.2469,128585,right,2,151.6812,196.0117,-54.95942,86.562950,-142.29400,-79.21784,...,-0.378559,181.7657,154.2318,-5.538349,280.1964,-30.71408,136.50200,0.000000,0.717604,0.0
40,336.3297,128595,right,2,161.3418,194.0368,-51.56314,101.533800,61.97456,88.98601,...,-0.210655,184.5448,161.6927,7.435162,274.6993,-20.33877,158.45080,0.210097,0.995388,0.0
50,336.4131,128604,right,2,165.5767,195.0496,-39.98222,-5.294424,-59.74868,158.58400,...,-0.129517,185.1178,168.2499,22.849030,289.2612,-3.87326,176.46150,0.499274,1.515088,0.0
60,336.4954,128614,right,2,164.2158,187.2864,-30.50245,-59.360860,-147.21790,38.27838,...,-0.150328,186.8537,167.3232,33.900690,341.7870,45.99316,193.09770,1.000000,2.642317,0.0
70,336.5782,128623,right,2,154.7117,169.2967,-29.78124,-222.415500,-325.78590,-70.71833,...,-0.203208,179.9163,154.1983,35.030910,324.7994,39.30713,207.88320,1.000000,2.802242,0.0
80,336.6611,128633,right,2,141.2121,145.9520,-41.24849,-61.384270,-152.02270,-184.45540,...,-0.326293,174.9414,142.3450,21.339400,355.9629,74.25906,184.63220,0.000000,0.513679,0.0
90,336.7439,128642,right,2,138.2178,137.4820,-54.98854,-52.558110,-115.59260,-175.16670,...,-0.382157,173.5485,131.5772,6.680511,326.8031,60.20363,195.06270,0.000000,0.162535,0.0


### cleaning the data

In [15]:
def clean(df):
    right = df[df[' Hand Type'] == 'right'].drop(' Hand Type', axis=1).drop(' # hands',axis=1 )
    left =  df[df[' Hand Type'] == 'left'].drop(' Hand Type', axis=1).drop(' # hands',axis=1 )
    new = pd.merge(right,left, on= ' Frame ID') #.drop('Time_x', axis=1).drop('Time_y',axis=1 )
    return new
 

In [26]:
sync1_new = clean(sync1)
sync1_new = sync1_new[::25]

In [27]:
sync1_new. head(20)

,Time_x,Frame ID,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,Pitch_x,Roll_x,...,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y
0,427.2310,139071,69.65445,209.9525,-18.547970,-65.047080,338.263100,-102.576300,0.865190,-1.009409,...,1.220096,-137.3886,158.1433,-27.829490,-261.8767,-47.873840,30.625840,0.0,0.000000,0.0
25,427.6452,139119,88.57839,301.6772,-21.715510,130.513000,-64.847360,88.459340,1.174654,-1.132521,...,1.109005,-159.4704,218.5509,4.723629,-317.2644,27.611140,4.083759,0.0,0.000000,0.0
50,428.0593,139166,133.64770,175.2845,59.926180,15.620420,-177.870400,158.678600,0.609984,-1.158887,...,1.017560,-155.9984,118.0704,69.887830,-346.0325,-25.602440,137.729800,0.0,0.000000,0.0
75,428.4735,139214,88.16529,203.5105,-3.703102,-106.265900,194.808100,-110.988700,1.202959,-1.220791,...,1.284278,-115.2666,114.4282,-18.583820,-315.7906,-20.881650,34.634790,0.0,0.000000,0.0
100,428.8878,139262,77.64143,276.7719,16.824770,44.574170,79.049160,142.447600,1.994686,-1.301623,...,2.468843,-133.3705,251.8106,-14.532300,-292.7469,63.630880,-37.735600,0.0,0.000000,0.0
125,429.3023,139310,86.92545,250.7338,47.926490,36.069100,-8.999642,-79.897690,2.258485,-1.312106,...,2.733876,-146.2523,152.6520,-24.671400,-303.9407,-36.543180,-50.946730,0.0,0.000000,0.0
150,429.7160,139357,133.10430,227.2561,-5.580800,76.642780,-150.922900,-16.217190,0.455563,-1.732109,...,-3.058041,-127.9536,184.9257,23.325470,-295.9757,7.617480,-19.370100,0.0,0.020727,0.0
175,430.1302,139405,136.73840,186.1687,-7.352259,-18.889910,30.973980,20.575400,0.214821,-1.945404,...,-0.197093,-111.8453,159.2111,38.588770,-254.9044,-32.076920,105.188500,0.0,0.174987,0.0
200,430.5443,139453,115.81060,246.0159,22.375600,-20.127660,266.990000,111.297100,1.099928,-1.351711,...,0.230211,-113.0468,160.6184,44.113510,-268.7853,-23.947030,100.448700,0.0,0.000000,0.0
225,430.9586,139500,103.61970,231.4475,73.438730,-19.819200,-185.468800,22.988230,2.034497,-1.263057,...,2.300608,-123.8968,182.1298,59.568430,-292.0663,2.518906,28.722200,0.0,0.000000,0.0


In [ ]:
sync1_new


In [108]:
sync1_new = clean(sync1)
sync2_new = clean(sync2)
sync3_new = clean(sync3)
sync4_new = clean(sync4)
sync5_new = clean(sync5)
sync6_new = clean(sync6)
sync7_new = clean(sync7)
sync8_new = clean(sync8)
sync9_new = clean(sync9)

sync1_new

,Frame ID,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,Pitch_x,Roll_x,Yaw_x,...,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y
0,139071,69.65445,209.9525,-18.54797,-65.047080,338.263100,-102.57630,0.865190,-1.009409,-0.763543,...,1.220096,-137.3886,158.1433,-27.82949,-261.8767,-47.87384,30.62584,0.0,0.0,0.0
1,139073,68.97833,215.1191,-20.12599,-44.440450,302.917000,-83.93305,0.863391,-1.005569,-0.766907,...,1.276068,-135.5462,162.8801,-30.98293,-259.9806,-43.46907,26.40578,0.0,0.0,0.0
2,139075,68.11068,220.9579,-21.77186,-22.080520,307.057400,-67.08289,0.875840,-1.017380,-0.767062,...,1.312464,-133.7321,167.7086,-33.75665,-257.4434,-37.24958,29.84401,0.0,0.0,0.0
3,139077,68.26217,226.7594,-22.93307,7.860982,335.314400,-61.81393,0.883429,-1.023885,-0.767406,...,1.338415,-133.4904,173.9064,-36.19991,-255.7929,-29.82336,33.75477,0.0,0.0,0.0
4,139078,68.29678,230.0778,-23.70698,3.900916,374.071900,-87.24055,0.890813,-1.033500,-0.761958,...,1.355570,-133.5924,177.2040,-37.22458,-254.6681,-26.72015,34.28068,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4521,147698,111.39700,182.9246,34.27466,-42.475920,-117.845800,-61.32522,0.395246,-1.971680,-0.640369,...,1.065970,-127.3144,194.4717,59.52803,-299.5664,15.22464,16.52840,0.0,0.0,0.0
4522,147700,110.08700,180.8389,32.92039,-71.148420,-111.439600,-69.28531,0.365014,-1.987410,-0.661939,...,1.007618,-126.9333,193.0801,58.90355,-298.9101,12.87815,18.89758,0.0,0.0,0.0
4523,147702,109.17140,179.3461,31.84416,-46.123690,-69.914710,-57.69822,0.355285,-1.995915,-0.671905,...,0.956325,-126.9241,191.1363,58.91284,-302.6545,12.99574,26.75648,0.0,0.0,0.0
4524,147704,108.90170,178.8129,31.02029,-15.531000,-5.445981,-50.80544,0.355584,-2.007045,-0.671661,...,0.901075,-126.4018,188.6597,59.13661,-308.9988,15.97083,37.08796,0.0,0.0,0.0


In [109]:
sync = pd.concat([sync1_new, sync2_new, sync3_new, sync4_new, sync5_new, sync6_new, sync7_new, sync8_new, sync9_new], axis=0)
sync

,Frame ID,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,Pitch_x,Roll_x,Yaw_x,...,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y
0,139071,69.65445,209.9525,-18.54797,-65.047080,338.26310,-102.57630,0.865190,-1.009409,-0.763543,...,1.220096,-137.3886,158.1433,-27.82949,-261.8767,-47.873840,30.62584,0.0,0.0,0.0
1,139073,68.97833,215.1191,-20.12599,-44.440450,302.91700,-83.93305,0.863391,-1.005569,-0.766907,...,1.276068,-135.5462,162.8801,-30.98293,-259.9806,-43.469070,26.40578,0.0,0.0,0.0
2,139075,68.11068,220.9579,-21.77186,-22.080520,307.05740,-67.08289,0.875840,-1.017380,-0.767062,...,1.312464,-133.7321,167.7086,-33.75665,-257.4434,-37.249580,29.84401,0.0,0.0,0.0
3,139077,68.26217,226.7594,-22.93307,7.860982,335.31440,-61.81393,0.883429,-1.023885,-0.767406,...,1.338415,-133.4904,173.9064,-36.19991,-255.7929,-29.823360,33.75477,0.0,0.0,0.0
4,139078,68.29678,230.0778,-23.70698,3.900916,374.07190,-87.24055,0.890813,-1.033500,-0.761958,...,1.355570,-133.5924,177.2040,-37.22458,-254.6681,-26.720150,34.28068,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,67304,97.75611,205.9629,30.04651,28.362980,77.44335,162.14280,0.909181,-1.253298,-0.495629,...,0.429763,-107.8500,127.8215,32.85250,-318.7630,20.751860,138.01250,0.0,0.0,0.0
2246,67306,98.93275,208.3153,34.85991,44.383930,71.36857,187.54080,0.983621,-1.239014,-0.525644,...,0.434664,-108.0805,128.1729,33.40391,-314.6744,14.308990,139.99090,0.0,0.0,0.0
2247,67309,101.91230,212.0264,42.53568,79.958100,113.41030,190.77140,1.065334,-1.225539,-0.574317,...,0.456099,-109.0018,129.5246,33.89856,-304.9600,-0.548136,142.01090,0.0,0.0,0.0
2248,67311,103.84750,214.7484,47.07082,61.117730,95.13531,155.74530,1.110287,-1.214715,-0.612753,...,0.459446,-109.7250,130.7078,34.03360,-295.5957,-14.978170,140.02630,0.0,0.0,0.0


In [ ]:
spontan2 = clean(spontan2)
spontan3 = clean(spontan3)
spontan4 = clean(spontan4)
spontan5 = clean(spontan5)
spontan6 = clean(spontan6)
spontan7 = clean(spontan7)
spontan8 = clean(spontan8)
spontan9 = clean(spontan9)